In [21]:
import os.path
import pandas as pd
import sqlalchemy
import math

from dateutil import tz

import datetime
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import gspread
from gspread.utils import ValueRenderOption

import boto3
import calendar

In [20]:
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Denver')
given_datetime = datetime.datetime(2024, 8, 2, 13, 35, 32, 149469)

# Calculating one hour before the given datetime
now = given_datetime - datetime.timedelta(hours=1)
now = now.replace(tzinfo=from_zone)
now = now.astimezone(to_zone)

In [22]:
# format date
date = now.date()
day = str(date.day)
month = date.month
year = date.year
days_in_month = str(calendar.monthrange(year,month)[1])
month = str(date.month)
year = str(date.year)[2:]

now = str(now)
now = now[:19]

first_day_of_month = datetime.date(date.year, date.month, 1)
date = date.strftime('%Y-%m-%d')
first_day_of_month = first_day_of_month.strftime('%Y-%m-%d')

In [23]:
db_username="master"
db_password="sYgcyr-domvoc-5qemvo"
db_endpoint="amprydb.cj7ver2tncka.us-east-2.rds.amazonaws.com"
db_name="amprydb"
db_port=5432

engine = sqlalchemy.create_engine(f'postgresql://{db_username}:{db_password}@{db_endpoint}:{db_port}/{db_name}')
metadata = sqlalchemy.MetaData()

In [25]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive"]

# The ID and range of a sample spreadsheet.
spreadsheet_id = '1__0MY8nG25R4fHxR0EX2XRjM0kuCKpCnYOJRxMgsLFo'

# gather credentials for google api calls
credentials_dict = {
    'token': "ya29.a0AXooCgs3K8hDbGnEhOPIOeTp4ZKp6mVN2SCNX-b64iwKOR80KYFd-Cf28y0jvhFZC2Ft3sqRNNbaJV0g-bKQazWDoMFQNm1hxZMqcY0f40RnuScOAhRJqikdB76E9A2QtiaSLZUf26t3HoWKaLPgZDVyQJa7-eLK5rnKaCgYKAfYSARESFQHGX2MiE5hFQgkmKUU39E239xEAnA0171",
    'refresh_token': "1//06EuZ60AQ5J8VCgYIARAAGAYSNwF-L9IrtxQ6pc0GI2lbnmdMvAjIqPWy6dWkb16XcYYeEd-oQ55iE852bWzExZD6A-JfcdZt9LI",
    'token_uri': "https://oauth2.googleapis.com/token",
    'client_id': "258748449675-apududjhadom52pe31f5qa10nq5lbjvo.apps.googleusercontent.com",
    'client_secret': "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",
    'scopes': SCOPES
}

# Create credentials from the dictionary
creds = Credentials.from_authorized_user_info(credentials_dict)
service = build("sheets", "v4", credentials=creds)
client = gspread.authorize(creds)
spreadsheet = client.open('Datawave')

# Specify the sheet name
worksheet_name = 'partners'

# Open the worksheet by its name
sheet = spreadsheet.worksheet(worksheet_name)

if str(day) == '1':
    try:
        source_spreadsheet = client.open("Datawave")
        source_sheet = source_spreadsheet.worksheet('partners')  # or use get_worksheet(index) if not the first sheet

        # Get the target spreadsheet
        target_spreadsheet = client.open("Datawave History")

        # Get the source sheet ID
        source_sheet_id = source_sheet.id

        # Copy the sheet to the target spreadsheet
        request = {
            'destinationSpreadsheetId': target_spreadsheet.id
        }

        response = service.spreadsheets().sheets().copyTo(
            spreadsheetId=source_spreadsheet.id,
            sheetId=source_sheet_id,
            body=request
        ).execute()

        # Get the new sheet ID from the response
        new_sheet_id = response['sheetId']

        if month == '1':
            month1 = '12'
            year1 = str(int(year) - 1)
        else:
            month1 = str(int(month) - 1)
            year1 = year

        # Optionally, you can rename the new sheet
        target_spreadsheet.get_worksheet_by_id(new_sheet_id).update_title(f"partners_{month1}_{year1}")
        print('done')
    except:
        print('Sheet already exists')

In [28]:
def update_sheet(service, spreadsheet_id, range_name, values):
    body = {
        'values': values
    }
    result = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption='USER_ENTERED', body=body).execute()
#     print('{0} cells updated.'.format(result.get('updatedCells')))

sheet_name = 'partners'  # Name of the sheet
start_cell = f'D1'  # Starting cell (A2 indicates the first column, second row)
range_name = f'{sheet_name}!{start_cell}'
update_sheet(service, spreadsheet_id, range_name, [['UPDATING']])